In [1]:
# --- Install dependencies ---
!pip -q install SpeechRecognition pydub gTTS soundfile yfinance transformers torch openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 47.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.6 MB/s eta 0:00:00


In [2]:
 #--- Imports ---
import os, io, time, random, datetime, re
from typing import Optional
import yfinance as yf
import pandas as pd
from transformers import pipeline

# ================================================================
# 1️⃣  Load / Record Audio
# ================================================================


In [3]:
AUDIO_PATH = "/content/drive/MyDrive/myquestion.wav"  # <-- Replace with your audio file path or upload via Colab Files
print("🎧 Set AUDIO_PATH to:", AUDIO_PATH)


🎧 Set AUDIO_PATH to: /content/drive/MyDrive/myquestion.wav



# ================================================================
# 2️⃣  Speech-to-Text (STT) — Whisper fallback to Google
# ================================================================


In [4]:
def transcribe(audio_path: str) -> str:
    """Convert speech → text using Whisper or SpeechRecognition."""
    try:
        import whisper
        print("Using Whisper for transcription...")
        model = whisper.load_model("base")
        result = model.transcribe(audio_path)
        return result.get("text", "").strip()
    except Exception:
        print("Fallback to Google SpeechRecognition...")
        import speech_recognition as sr
        r = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)
        return r.recognize_google(audio)

print("🗣️ Transcription function ready.")


🗣️ Transcription function ready.


# ================================================================
# 3️⃣  Project Model — Real-time Stock + Sentiment Q&A
# ================================================================


In [5]:

# Load small Hugging Face sentiment model
sentiment_analyzer = pipeline("sentiment-analysis")

def run_llm(prompt: str) -> str:
    """Simple AI agent that handles live stock data, trends, and sentiment."""
    prompt = prompt.lower().strip()

    # Tickers dictionary
    tickers = {
        "apple": "AAPL",
        "microsoft": "MSFT",
        "google": "GOOGL",
        "amazon": "AMZN",
        "tesla": "TSLA"
    }

    # --- Live Stock Price ---
    for name, symbol in tickers.items():
        if name in prompt and ("price" in prompt or "value" in prompt):
            try:
                data = yf.Ticker(symbol)
                price = data.history(period="1d")["Close"].iloc[-1]
                return f"{name.title()}'s latest stock price is about ${price:.2f}."
            except Exception as e:
                return f"Couldn't fetch {name}'s price: {e}"

    # --- 5-Day Trend ---
    for name, symbol in tickers.items():
        if name in prompt and "trend" in prompt:
            try:
                data = yf.download(symbol, period="5d", interval="1d", progress=False)
                trend = data["Close"].iloc[-1] - data["Close"].iloc[0]
                direction = "upward 📈" if trend > 0 else "downward 📉"
                return f"In the past 5 days, {name.title()} shows a {direction} trend of {trend:.2f} USD."
            except Exception as e:
                return f"Trend check failed for {name}: {e}"

    # --- News Sentiment ---
    for name, symbol in tickers.items():
        if name in prompt and "sentiment" in prompt:
            try:
                data = yf.Ticker(symbol)
                news = data.news[:5]  # latest 5 headlines
                headlines = [n["title"] for n in news]
                sentiments = sentiment_analyzer(headlines)
                pos = sum(1 for s in sentiments if s["label"] == "POSITIVE")
                neg = sum(1 for s in sentiments if s["label"] == "NEGATIVE")
                overall = "positive 😊" if pos > neg else "negative 😟"
                return f"Recent news sentiment for {name.title()} appears {overall} ({pos} positive vs {neg} negative)."
            except Exception as e:
                return f"Couldn't analyze sentiment for {name}: {e}"

    # --- Generic / Fallback ---
    replies = {
        "forecast": "Overall sentiment is cautiously optimistic for this week.",
        "summary": "Tech stocks show mild positive momentum; energy remains neutral.",
        "agent": "This local AI agent integrates live data and sentiment from Yahoo Finance."
    }
    for key, val in replies.items():
        if key in prompt:
            return val

    return random.choice([
        "The market remains steady with moderate daily changes.",
        "Investors are focusing on upcoming earnings reports.",
        "Volatility is slightly below average today."
    ])

print("🤖 Project model ready.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


🤖 Project model ready.


# ================================================================
# 4️⃣  Text-to-Speech (TTS) — using gTTS
# ================================================================


In [6]:
def synthesize_tts(text: str, out_path="reply.mp3") -> str:
    """Convert text → speech and save to MP3."""
    try:
        from gtts import gTTS
        gTTS(text).save(out_path)
        return out_path
    except Exception as e:
        print("TTS failed:", e)
        return ""

print("🔊 TTS ready.")


🔊 TTS ready.



# ================================================================
# 5️⃣  End-to-End Demo
# ================================================================


In [7]:
try:
    print("🎬 Starting end-to-end demo...")
    text = transcribe(AUDIO_PATH)
    print("USER:", text)

    answer = run_llm(text)
    print("MODEL:", answer)

    synth_path = synthesize_tts(answer)
    print("✅ Audio reply saved to:", synth_path)

    # Optional: play audio inside Colab
    from IPython.display import Audio
    display(Audio(synth_path, autoplay=True))

except Exception as e:
    print("Demo error:", e)

🎬 Starting end-to-end demo...
Using Whisper for transcription...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 268MiB/s]


USER: What's the current price of Apple stock?
MODEL: Apple's latest stock price is about $258.21.
✅ Audio reply saved to: reply.mp3
